# Average return multiplied by  the average trade sign

$$ r_{i} (t,\tau) = \log m_{i}(t + \tau) - \log m_{i}(t) = \log \frac{m_{i}(t + \tau)}{m_{i}(t)} $$
$$ R_{ij}^{(exc. 0)}(\tau) \ne \left \langle r_{i}\left ( t, \tau \right ) \right \rangle \left \langle \epsilon_{j} \left ( t \right ) \right \rangle $$

Implementation of the right side of the last equation

In [1]:
# Import modules

import numpy as np
import pickle

from itch_data_generator import trade_sign_reshape

In [2]:
# Using the only two common stocks in both data sets
ticker_i = 'AAPL'
ticker_j = 'MSFT'
day = '09'
tau_val = 1000
t_step = 100

In [3]:
def avg_return_avg_trade_prod_data(ticker_i, ticker_j, day, tau_val, t_step):
    """
    Obtain the result of the product between the averaged midpoint log return
    of ticker i and the averaged trade signs of ticker j during different time
    lags. The data is adjusted to use only the values each t_step ms
        :param ticker_i: string of the abbreviation of the midpoint stock to
         be analized (i.e. 'AAPL')
        :param ticker_j: string of the abbreviation of the trade sign stock
         to be analized (i.e. 'AAPL')
        :param day: string of the day to be analized (i.e '07')
        :param tau_val: maximum time lag to be analyzed
        :param t_step: time step in the data in ms
    """

    print('Product between the averaged midpoint log return of ticker i and '
          + 'the averaged trade signs of ticker j data')
    print('Processing data for the stock i ' + ticker_i + ' and stock j '
          + ticker_j + ' the day ' + day + ' March, 2016')
    print('Time step: ', t_step, 'ms')

    # Load data
    midpoint_i = pickle.load(open(
        '../Data/midpoint_data/midpoint_201603{}_{}.pickl'
        .format(day, ticker_i), 'rb'))
    trade_sign_j = pickle.load(open(
        '../Data/trade_signs_data/trade_signs_most_201603{}_{}.pickl'
        .format(day, ticker_j), 'rb'))
    time = pickle.load(open('../Data/midpoint_data/time.pickl', 'rb'))

    # Setting variables to work with t_step ms accuracy

    # Array of the average of each tau. 10^3 s used by Wang
    avg_return_sign = np.zeros(tau_val)

    # Using values each second
    midpoint_i_sec = midpoint_i[::t_step]
    # Changing time from 1 ms to t_step ms
    time_t_step = time[::t_step]

    # reshape and average data of trade signs
    trade_sign_j_sec_avg, trade_sign_j_sec_nr = trade_sign_reshape(trade_sign_j, time_t_step)

    # Calculating the midpoint log return and the cross response functions

    for tau in range(1, tau_val):

        # Every second have a log-return
        log_return_i_sec = 0. * time_t_step

        # Obtain the midpoint log return. Displace the numerator tau values
        # to the right and compute the return, and append the remaining values
        # of tau with zeros
        log_return_i_sec = np.append(np.log(
                    midpoint_i_sec[tau:]/midpoint_i_sec[:-tau]), np.zeros(tau))

        avg_return_sign[tau] = (np.mean(
                                log_return_i_sec[trade_sign_j_sec_nr != 0]) *
                                np.mean(
                                trade_sign_j_sec_avg[trade_sign_j_sec_nr != 0]
                                ))

    # Saving data

    if (not os.path.isdir('../Data/avg_return_sign_data_{}ms/'
                          .format(t_step))):

        os.mkdir('../Data/avg_return_sign_data_{}ms/'.format(t_step))
        print('Folder to save data created')

    pickle.dump(avg_return_sign, open(
        '../Data/avg_return_sign_data_{}ms/avg_201603{}_{}i_{}j_{}ms.pickl'
        .format(t_step, day, ticker_i, ticker_j, t_step), 'wb'))

    print('Average product data saved')
    print()

    return None

In [ ]:
plt.figure(figsize=(16,9))
plt.subplots_adjust(hspace = 0, wspace = 0)

ax1 = plt.subplot(221)
plot1 = plt.semilogx(cross1, '-g', label='Stock i %s and stock j %s' % (ticker_i, ticker_j))
plt.xlabel(r'Time lag $[\tau]$', fontsize=15)
plt.ylabel(r'$ \left \langle r_{i}\left ( t, \tau \right ) \right \rangle \left \langle \epsilon_{j} \left ( t \right ) \right \rangle $', fontsize=15)
plt.legend(loc='best', fontsize=20)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.grid(True)

ax2 = plt.subplot(222)
plot2 = plt.semilogx(cross2, '-g', label='Stock i %s and stock j %s' % (ticker_j, ticker_i))
plt.xlabel(r'Time lag $[\tau]$', fontsize=15)
plt.ylabel(r'$ \left \langle r_{i}\left ( t, \tau \right ) \right \rangle \left \langle \epsilon_{j} \left ( t \right ) \right \rangle $', fontsize=15)
plt.legend(loc='best', fontsize=20)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.grid(True)

ax3 = plt.subplot(223)
plot3 = plt.semilogx(cross3, '-g', label='Stock i %s and stock j %s' % (ticker_i, ticker_i))
plt.xlabel(r'Time lag $[\tau]$', fontsize=15)
plt.ylabel(r'$ \left \langle r_{i}\left ( t, \tau \right ) \right \rangle \left \langle \epsilon_{j} \left ( t \right ) \right \rangle $', fontsize=15)
plt.legend(loc='best', fontsize=20)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.grid(True)

ax4 = plt.subplot(224)
plot4 = plt.semilogx(cross4, '-g', label='Stock i %s and stock j %s' % (ticker_j, ticker_j))
plt.xlabel(r'Time lag $[\tau]$', fontsize=15)
plt.ylabel(r'$ \left \langle r_{i}\left ( t, \tau \right ) \right \rangle \left \langle \epsilon_{j} \left ( t \right ) \right \rangle $', fontsize=15)
plt.legend(loc='best', fontsize=20)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.grid(True)

plt.tight_layout()

#plt.savefig('avg_product.png')